In this notebook, we will analyse the profitability of traders
- uses summary_profitability.csv

Analysis method used;
- `profitability.py` is used to generate required data for this analysis
- the above script requires `tools.csv` which is extracted using `tools.py`
- `profitability.py` makes two CSVs
    - `all_trades_profitability.csv` a csv analysing all the trades
    - `summary_profitability.csv` a csv summarizing the above for each trader
- in the analysis, only `closed` market trades are analysed (everything else is skipped)

In [26]:
import pandas as pd
import matplotlib.pyplot as plt

pd.set_option('display.precision', 4)
plt.style.use('ggplot')

In [28]:
all_trades = pd.read_csv('./data/all_trades_profitability.csv')
summary = pd.read_csv('./data/summary_profitability.csv')

# select only traders who have traded at least 10 times
summary = summary[summary['num_trades'] >= 15]

In [29]:
all_trades.columns

Index(['trader_address', 'trade_id', 'creation_timestamp', 'title',
       'market_status', 'collateral_amount', 'outcome_index',
       'trade_fee_amount', 'outcomes_tokens_traded', 'current_answer',
       'is_invalid', 'winning_trade', 'earnings', 'redeemed',
       'redeemed_amount', 'num_mech_calls', 'mech_fee_amount', 'net_earnings',
       'roi'],
      dtype='object')

In [30]:
summary.columns

Index(['trader_address', 'num_trades', 'num_winning_trades', 'num_redeemed',
       'total_investment', 'total_trade_fees', 'num_mech_calls',
       'total_mech_fees', 'total_earnings', 'total_redeemed_amount',
       'total_net_earnings', 'total_roi', 'mean_mech_calls_per_trade',
       'mean_mech_fee_amount_per_trade', 'total_net_earnings_wo_mech_fees',
       'total_roi_wo_mech_fees'],
      dtype='object')

In [31]:
# number of traders
print(f"Number of traders: {len(summary)}")

Number of traders: 93


In [32]:
# Number of trades
print(f"Number of trades: {all_trades['trade_id'].nunique():,}")

Number of trades: 36,190


In [33]:
# Stats of trades per trader
summary['num_trades'].quantile([0.25, 0.5, 0.75, 0.9, 0.95, 0.99])
# Median trader makes about 236 trades

0.25      51.0
0.50     236.0
0.75     509.0
0.90     846.2
0.95    1318.6
0.99    2400.4
Name: num_trades, dtype: float64

In [34]:
# Stats of num_mech_calls per trader
summary['num_mech_calls'].quantile([0.25, 0.5, 0.75, 0.9, 0.95, 0.99])

0.25      134.0
0.50      548.0
0.75     1669.0
0.90     3869.0
0.95    10396.4
0.99    20135.4
Name: num_mech_calls, dtype: float64

In [35]:
# Stats of num_mech_calls per trade - median makes 3 mech calls per trade
all_trades['num_mech_calls'].quantile([0.25, 0.5, 0.75, 0.9, 0.95, 0.99])

0.25     1.0
0.50     3.0
0.75     4.0
0.90     8.0
0.95    12.0
0.99    49.0
Name: num_mech_calls, dtype: float64

In [36]:
# Stats of winning_trades per trader
summary['num_winning_trades'].apply(lambda x: int(x)).quantile([0.25, 0.5, 0.75, 0.9, 0.95, 0.99])

0.25      25.00
0.50     103.00
0.75     218.00
0.90     382.80
0.95     596.40
0.99    1122.04
Name: num_winning_trades, dtype: float64

In [37]:
summary['perc_winning_trades'] = summary['num_winning_trades'].map(int) / summary['num_trades']

# Stats of winning_trades per trade
summary['perc_winning_trades'].quantile([0.25, 0.5, 0.75, 0.9, 0.95, 0.99])

0.25    0.4197
0.50    0.4545
0.75    0.4794
0.90    0.5372
0.95    0.5677
0.99    0.6858
Name: perc_winning_trades, dtype: float64

In [38]:
# stats on net earnings per trader
summary['total_net_earnings'].quantile([0.25, 0.5, 0.6, 0.7, 0.75, 0.9, 0.95, 0.99])

0.25   -23.3910
0.50    -7.4054
0.60    -4.9755
0.70    -2.7644
0.75    -1.6614
0.90    -0.1711
0.95     1.5745
0.99    87.8129
Name: total_net_earnings, dtype: float64

In [39]:
# stats on net earnings per trader without mech calls
summary['total_net_earnings_wo_mech_fees'].quantile([0.25, 0.5, 0.6, 0.7, 0.75, 0.9, 0.95, 0.99])

0.25    -4.7586
0.50    -1.9011
0.60    -0.9722
0.70    -0.3899
0.75    -0.1381
0.90     1.6991
0.95     4.6127
0.99    90.8397
Name: total_net_earnings_wo_mech_fees, dtype: float64

In [40]:
# stats on average collateral per trade. Seems very low considering per mech call fee of 0.01 (and per trade fee of 2%)
all_trades['collateral_amount'].quantile([0.25, 0.5, 0.6, 0.7, 0.75, 0.9, 0.95, 0.99])

# taking 50th percentile of collateral amount per trade
# each trader loses about 50% of their collateral per trade on fees (assuming only one mech call)

0.25    0.0329
0.50    0.0600
0.60    0.0800
0.70    0.1000
0.75    0.1000
0.90    0.1208
0.95    0.2384
0.99    2.2200
Name: collateral_amount, dtype: float64

In [41]:
# stats on ROI per trade
all_trades['roi'].quantile([0.25, 0.5, 0.6, 0.7, 0.75, 0.9, 0.95, 0.99])

0.25   -1.5450
0.50   -1.1450
0.60   -0.5518
0.70    0.1862
0.75    0.3756
0.90    0.6839
0.95    0.7770
0.99    0.9133
Name: roi, dtype: float64

In [42]:
# stats on ROI per trader
summary['total_roi'].quantile([0.25, 0.5, 0.6, 0.7, 0.75, 0.9, 0.95, 0.99])

0.25   -0.7403
0.50   -0.4751
0.60   -0.4227
0.70   -0.3262
0.75   -0.3038
0.90   -0.0370
0.95    0.1351
0.99    0.3781
Name: total_roi, dtype: float64